In [1]:
from cv2 import line, putText, FONT_HERSHEY_SIMPLEX
import numpy as np


def draw_tracks(sequence : np.array, tracks_data : np.array) -> np.array:
    '''
    Function that draw the tracks given the sequence and the array of tracks
    
    Input:
        - squence: array of shape (N,H,W,C)
            N: number of frames
            H: Height
            W: Width
            C: Channel (empty if grayscale)
        - tracks_data: array where:
                            tracks_data[0] track_id
                            tracks_data[1] x
                            tracks_data[2] y
                            tracks_data[3] fluorescence
                            tracks_data[4] frame
    
    Ouput:
        - seq_out: array of shape (N,H,W,C) 
    '''
    frames, H, W = sequence.shape[0:3]
    seq_out = sequence.copy()
    
    if len(sequence.shape) < 4:
        seq_out = np.zeros((frames, H, W, 3))
        seq_out[:, :, :, 0] = sequence.copy()
        
    tracks_id = (np.unique(tracks_data[:, 0])).tolist()
    
    font                   = FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,500)
    fontScale              = 0.9
    fontColor              = (255,255,255)
    lineType               = 1

    
    tracks = []
    for track_id in tracks_id:
        data_track = tracks_data[tracks_data[:, 0]==track_id]
        coord = np.ones((frames+1, 4)) * -1
        coord[np.uint(data_track[:, 4])] = data_track[:, 0:4]
        tracks.append(coord)
    colors = np.random.uniform(100, 255, (len(tracks_id), 3))
    
    for frame in range(frames):
        start = 1
        if frame > 20:
            start = frame - 20
        for i in range(start,frame+1):
            
            for tr, track in enumerate(tracks):
                if track[i,1]>0 and track[i-1,1]>0:
                    line(seq_out[frame], (int(track[i-1,1]),int(track[i-1,2]) ), (int(track[i,1]),int(track[i,2]) ), colors[tr], thickness=3)
            
                    fontColor = colors[tr]
                    coord = (int(track[frame,2]) + 5,int(track[frame,1]) + 5)
                    '''putText(seq_out[frame],'ID: %d, F : %d' % (track[frame,0], track[frame,3]), 
                            coord, 
                            font, 
                            fontScale,
                            fontColor,
                            lineType)'''
                    
    return np.uint8(seq_out)


In [30]:
tracks_id = (np.unique(data[:, 0])).tolist()
track_id = 3

data_track = data[data[:, 0]==track_id]
print(data_track)
print(data_track[:, 4])
print(data_track[:, 0:4])

[[  3.         372.71428571   3.14285714          nan   2.        ]
 [  3.         372.42857143   3.28571429          nan   3.        ]
 [  3.         372.16666667   3.16666667          nan   4.        ]
 [  3.         372.125        2.875               nan   5.        ]
 [  3.         372.125        2.875               nan   6.        ]
 [  3.         372.125        2.875               nan   7.        ]
 [  3.         372.125        2.875               nan   8.        ]
 [  3.         372.125        2.875               nan   9.        ]
 [  3.         372.           3.                  nan  10.        ]
 [  3.         372.           3.                  nan  11.        ]
 [  3.         372.           3.                  nan  12.        ]
 [  3.         372.           3.                  nan  13.        ]
 [  3.         372.           3.                  nan  14.        ]
 [  3.         372.           3.                  nan  15.        ]
 [  3.         371.8          2.6               

In [3]:
import imageio
import pandas as pd
import numpy as np
from imageio import mimwrite as mp4_writer

sequence = imageio.mimread('1062 semen sin trayectorias-01.avi')  # importar video
asequence = np.array(sequence)

#recortar = sequence[:,200:350,350:500,:]

data = pd.read_csv('tracks-01.csv')
data = (data.to_numpy())[:, 0:5]

seq_tracks = draw_tracks(asequence, data)
mp4_writer("draw-tracks-01.mp4", seq_tracks, format="mp4", fps=1)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (780, 580) to (784, 592) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
